In [79]:
%matplotlib inline
from tqdm import tqdm
import numpy as np
import pymc3 as pm
np.set_printoptions(suppress=True)

In [ ]:
# Parameters
N = 100000 # True total individuals
S = 10  # Sample periods

# Arrival: 40% initially present
birth = np.concatenate(([.4], np.ones(S-1) * (1-.4)/(S-1)), axis=0) 

# Departure: fixed 20% rate All depart at 
dep  = np.concatenate((np.ones(S-1) * .2, [1.]), axis=0)

# Capture probability (propensity)
alpha = .25
beta = 5
prop = pm.Beta.dist(alpha, beta).random(size = N)

In [130]:
# Capture History Data Structures
CH = [] # Full CH matrix
GCH = [] # First/Last/Count CH matrix
TH = []

# Generate and Arrival Distribution lmd ~ Cat(arr)
Arrival_DIST = pm.Categorical.dist(birth)

for i in tqdm(range(N)):
    arrival = Arrival_DIST.random() # We make sure that the individual is assigned to an arrival
    dep_tmp = dep.copy()
    dep_tmp[:arrival] = 0
    dep_tmp = np.append([1],(1-dep_tmp).cumprod())[:-1] * dep_tmp # Based on the arrival, compute probs of departure
    departure = pm.Categorical.dist(dep_tmp).random() # Sample departure
    dur = departure - arrival + 1
    capture_history = pm.Bernoulli.dist(prop[i]).random(size=(dur,1))
    capture_history = np.concatenate((np.zeros(arrival), 
                                      np.array(capture_history), 
                                      np.zeros(S - departure - 1)), 
                                     axis = 0)
    TH.append([int(arrival), int(departure), dur, prop[i], capture_history.sum()])
    if capture_history.any():
        first = capture_history.argmax()
        last = S - capture_history[::-1].argmax() -1
        count = capture_history.sum()
        GCH.append([first, last, count])
        CH.append(capture_history)

100%|██████████| 1000/1000 [00:27<00:00, 36.42it/s]


In [119]:
import pickle
fname = 'data/gch-%d-%d-%.2f-%.2f.pkl' % (N, S, alpha, beta)
fname
with open(fname, 'wb') as f:
  pickle.dump((GCH, sorted(TH, key=lambda tup: tup[3])), f)
fname

'data/gch-100-10-0.25-5.00.pkl'

In [122]:
np.sort(prop.round(3))

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.002, 0.002, 0.002, 0.002, 0.002, 0.003, 0.003,
       0.003, 0.003, 0.004, 0.004, 0.004, 0.004, 0.005, 0.006, 0.006,
       0.007, 0.007, 0.007, 0.009, 0.009, 0.011, 0.012, 0.012, 0.015,
       0.017, 0.017, 0.018, 0.019, 0.022, 0.024, 0.026, 0.028, 0.028,
       0.03 , 0.034, 0.036, 0.037, 0.038, 0.039, 0.04 , 0.047, 0.055,
       0.062, 0.067, 0.067, 0.072, 0.076, 0.077, 0.08 , 0.089, 0.091,
       0.099, 0.101, 0.114, 0.116, 0.121, 0.127, 0.188, 0.244, 0.246,
       0.248, 0.26 , 0.272, 0.311, 0.324, 0.326, 0.33 , 0.335, 0.355,
       0.358])

In [123]:
birth.shape

(10,)

In [126]:
len(GCH)

7